In [1]:
# Argparse
import argparse
opt = argparse.Namespace( # Fake parsed arguments
    seed=7,
    device=-1,
    cuda=False,
    visdom=True
)

# Util
import cdae.util as util
util.init(opt)

# Distributions
import cdae.distributions as dists

# Torch
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data
import torch.optim as optim

# Plotting
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.available
plt.style.use('seaborn-whitegrid')

%matplotlib notebook

# Generative Model

The true generative model $p$ over latents $k, x$ and observed variables $y$ is:
\begin{align}
    k &\sim \mathrm{Categorical}([10, 20], [0.5, 0.5]) \\\
    x &\sim \mathrm{Normal}(0, 1) \\\
    y &\sim \mathrm{Normal}(f(k, x) ,1)
\end{align}
where $f: \mathbb R^2 \to \mathbb R$ is defined as
\begin{align}
    f(k, x) = a(k + x)^2 + b(k + x) + c.
\end{align}


In [2]:
def f(k, x, a, b, c):
    return a * (k + x)**2 + b * (k + x) + c

def generative_model(a, b, c):
    k = np.random.choice([10, 20], p=[0.5, 0.5])
    x = np.random.normal(loc=0, scale=1)
    y = np.random.normal(f(k, x, a, b, c), 1)
    
    return y

(a, b, c) = (1, 0, 0)
num_data = 1000
data = np.array([generative_model(a, b, c) for i in range(num_data)])
fig, ax = plt.subplots()
sns.kdeplot(data, ax=ax)
ax.set_title('Kernel density estimate of $p(y)$')
ax.set_xlabel('$y$')
ax.set_ylabel('$p(y)$')

<IPython.core.display.Javascript object>

# Generative Network

Assume that we actually don't know the form of $f$ and that we want to learn it, i.e. the true model $p$ from a dataset $(y^{(n)})_{n = 1}^N$.
Let's model the family of functions $f$ under consideration as a neural network parameterized by generative weights $\theta$ such that it maps from $\mathbb R^2$ to $\mathbb R$:

In [3]:
class GenerativeNetwork(nn.Module):
    def __init__(self):
        '''
        Initialize generative network.
        '''
        super(GenerativeNetwork, self).__init__()
        self.lin1 = nn.Linear(2, 16)
        self.lin2 = nn.Linear(16, 1)
        
        init.xavier_uniform(self.lin1.weight, gain=init.calculate_gain('relu'))
        init.xavier_uniform(self.lin2.weight)
        
    def f_approx(self, k, x):
        '''
        Returns output of current approximation of f.
        
        input:
            k: Variable [batch_size, 1]
            x: Variable [batch_size, 1]

        output: Variable [batch_size, 1]
        '''
        
        ret = self.lin1(torch.cat([k, x], dim=1))
        ret = F.relu(ret)
        ret = self.lin2(ret)
        
        return ret

    def forward(self, k, x, y):
        '''
        Returns log p_{\theta}(k, x, y)
        
        input:
            k: Variable [batch_size, 1]
            x: Variable [batch_size, 1]
            y: Variable [batch_size, 1]
            
        output: Variable [batch_size, 1]
        '''
        
        batch_size = k.size(0)
        
        logpdf_k = dists.categorical_logpdf(
            k, 
            categories=Variable(torch.Tensor([10, 20]).unsqueeze(-1).unsqueeze(-1).expand(2, batch_size, 1)),
            probabilities=Variable(torch.Tensor([0.5, 0.5]).unsqueeze(-1).unsqueeze(-1).expand(2, batch_size, 1))
        )
        
        logpdf_x = dists.normal_logpdf(x, Variable(torch.zeros(x.size())), Variable(torch.ones(x.size())))

        mean = self.f_approx(k, x)
        var = Variable(torch.ones(mean.size()))
        logpdf_y = dists.normal_logpdf(y, mean, var)
        
        return logpdf_k + logpdf_x + logpdf_y
    
    def sample(self, batch_size):
        '''
        Returns sample from the generative model.
        
        input:
            batch_size: int
        
        output:
            k: Tensor [batch_size, 1]
            x: Tensor [batch_size, 1]
            y: Tensor [batch_size, 1]
        '''
        
        k = dists.categorical_sample(
            categories=torch.Tensor([10, 20]).unsqueeze(-1).unsqueeze(-1).expand(2, batch_size, 1),
            probabilities=torch.Tensor([0.5, 0.5]).unsqueeze(-1).unsqueeze(-1).expand(2, batch_size, 1)
        )
        x = dists.normal_sample(
            mean=torch.zeros(batch_size, 1),
            var=torch.ones(batch_size, 1)
        )
        
        mean = self.f_approx(Variable(k, volatile=True), Variable(x, volatile=True)).data
        var = torch.ones(mean.size())
        y = dists.normal_sample(
            mean=mean,
            var=var
        )
        
        return k, x, y
    
    def plot_y_density(self, num_data, ax=None):
        '''
        Returns an Axes object with the kernel density estimate (KDE) of p_{\theta}(y).
        
        input:
            num_data: int. number of data points to make the KDE plot
            ax: (optional) Axes object
            
        output: update ax.
        '''
        
        if ax is None:
            _, ax = plt.subplots()

        _, _, data = self.sample(num_data)
        data = data.view(-1).numpy()

        sns.kdeplot(data, ax=ax)
        ax.set_title('Kernel density estimate of $p_{\\theta}(y)$')
        ax.set_xlabel('$y$')
        ax.set_ylabel('$p_{\\theta}(y)$')

In [4]:
generative_network = GenerativeNetwork()
num_data = 1000
generative_network.plot_y_density(num_data)

<IPython.core.display.Javascript object>

# Inference Network

We seek to learn an inference network $q_{\phi}(k, x \lvert y)$ parameterized by $\phi$ which, given $y$ maps to the parameters of the distribution over $(k, x)$, ideally close to the posterior under the true model, $p(k, x \lvert y)$.

Let
\begin{align}
    q_{\phi}(k, x \lvert y) &= q_{\phi}(k \lvert y) q_{\phi}(x \lvert k, y) \\\
    q_{\phi}(k \lvert y) &= \mathrm{Categorical}([10, 20], [\phi_1, \phi_2]) \\\
    q_{\phi}(x \lvert k, y) &= \mathrm{Normal}(\phi_3, \phi_4)
\end{align}
where $\phi = [\phi_1, \dotsc, \phi_4]$ is the output of the inference network.

In [5]:
class InferenceNetwork(nn.Module):
    def __init__(self):
        '''
        Initialize inference network.
        '''
        
        super(InferenceNetwork, self).__init__()
        self.k_lin1 = nn.Linear(1, 16)
        self.k_lin2 = nn.Linear(16, 2)
        
        self.x_mean_lin1 = nn.Linear(2, 16)
        self.x_mean_lin2 = nn.Linear(16, 1)
        
        self.x_var_lin1 = nn.Linear(2, 16)
        self.x_var_lin2 = nn.Linear(16, 1)

        init.xavier_uniform(self.k_lin1.weight, gain=init.calculate_gain('relu'))
        init.xavier_uniform(self.k_lin2.weight)
        init.xavier_uniform(self.x_mean_lin1.weight, gain=init.calculate_gain('relu'))
        init.xavier_uniform(self.x_mean_lin2.weight)
        init.xavier_uniform(self.x_var_lin1.weight, gain=init.calculate_gain('relu'))
        init.xavier_uniform(self.x_var_lin2.weight)
        
    def get_q_k_params(self, y):
        '''
        Returns parameters \phi_1, \phi_2.
        
        input:
            y: Variable [batch_size, 1]
            
        output: Variable [batch_size, 2]
        '''
        
        ret = self.k_lin1(y)
        ret = F.relu(ret)
        ret = self.k_lin2(ret)
        ret = F.softmax(ret)
        
        return ret
    
    def get_q_x_params(self, k, y):
        '''
        Returns parameters \phi_3, \phi_4.
        
        input:
            k: Variable [batch_size, 1]
            y: Variable [batch_size, 1]
            
        output:
            mean: Variable [batch_size, 1]
            var: Variable [batch_size, 1]
        '''
        
        mean = self.x_mean_lin1(torch.cat([k, y], dim=1))
        mean = F.relu(mean)
        mean = self.x_mean_lin2(mean)
        
        var = self.x_var_lin1(torch.cat([k, y], dim=1))
        var = F.relu(var)
        var = self.x_var_lin2(var)
        var = F.softplus(var)
        
        return mean, var
        
    def forward(self, k, x, y):
        '''
        Returns log q_{\phi}(k, x | y)
        
        input:
            k: Variable [batch_size, 1]
            x: Variable [batch_size, 1]
            y: Variable [batch_size, 1]
            
        output: Variable [batch_size, 1]
        '''
        batch_size, _ = k.size()
        
        probabilities = self.get_q_k_params(y)
        logpdf_k = dists.categorical_logpdf(
            k,
            categories=Variable(torch.Tensor([10, 20]).unsqueeze(-1).unsqueeze(-1).expand(2, batch_size, 1)),
            probabilities=torch.t(probabilities).unsqueeze(-1)
        )
        
        mean, var = self.get_q_x_params(k, y)
        logpdf_x = dists.normal_logpdf(
            x,
            mean=mean,
            var=var
        )
        
        return logpdf_k + logpdf_x
    
    def sample(self, y):
        '''
        Returns samples from q_{\phi}(k, x | y)
        
        input:
            y: Tensor [batch_size, 1]

        output:
            k: Tensor [batch_size, 1]
            x: Tensor [batch_size, 1]
        '''
        
        batch_size = y.size(0)
        
        probabilities = self.get_q_k_params(Variable(y, volatile=True)).data
        k = dists.categorical_sample(
            categories=torch.Tensor([10, 20]).unsqueeze(-1).unsqueeze(-1).expand(2, batch_size, 1),
            probabilities=torch.t(probabilities).unsqueeze(-1)
        )
        
        mean, var = self.get_q_x_params(Variable(k, volatile=True), Variable(y, volatile=True))
        x = dists.normal_sample(
            mean=mean.data,
            var=var.data
        )
        
        return k, x

In [6]:
inference_network = InferenceNetwork()
(a, b, c) = (1, 0, 0)
y_single = generative_model(a, b, c)
y = torch.Tensor([[y_single]]).expand(1000, 1)
k, x = inference_network.sample(y)
average_q_logpdf = torch.mean(
    inference_network.forward(
        Variable(k, volatile=True),
        Variable(x, volatile=True), 
        Variable(y, volatile=True)
    )
)
print('average_q_logpdf')
print(average_q_logpdf.data[0])

print('y')
print(y_single)

fig, ax = plt.subplots(nrows=1, ncols=2)
fig.set_size_inches(10, 4)

sns.distplot(k.view(-1).numpy(), ax=ax[0], kde=False)
ax[0].set_title('Histogram of $q_{\phi}(k | y)$')
ax[0].set_xlabel('$k$')
ax[0].set_ylabel('$q_{\phi}(k | y)$ for initial $\phi$')

sns.kdeplot(x.view(-1).numpy(), ax=ax[1])
ax[1].set_title('Kernel density estimate of $q_{\phi}(x | y)$')
ax[1].set_xlabel('$x$')
ax[1].set_ylabel('$q_{\phi}(x | y) = \int_k q_{\phi}(k, x | y) \,dk$ for initial $\phi$')

average_q_logpdf
-3.9244086742401123
y
148.70438056317417


<IPython.core.display.Javascript object>

# Training Loop

In [7]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, infinite_data=None, num_data=None, data_generator=None):
        '''
        Initializes SimpleDataset. If infinite_data is True, generates data on the fly,
        otherwise generates data once at the start.
        
        input:
            infinite_data: bool. If True, supply fake_num_data and data_generator, otherwise supply data
            num_data: number. In the case of infinite_data, this forms as a fake num_data in order to be able to talk about "epochs".
            data_generator: function that generates a sample from the true generative model
        '''
        assert(type(infinite_data) is bool)
        assert(type(num_data) is int)
        assert(callable(data_generator))
        
        self.infinite_data = infinite_data
        if infinite_data:
            self.num_data = num_data
            self.data_generator = lambda: np.float32(data_generator())
        else:
            self.num_data = num_data
            self.data = np.array([np.float32(generative_model(a, b, c)) for i in range(num_data)])
    
    def __len__(self):
        return self.num_data
    
    def __getitem__(self, index):
        if self.infinite_data:
            return self.data_generator()
        else:
            return self.data[index]

In [8]:
inference_network = InferenceNetwork()
generative_network = GenerativeNetwork()

inference_network_optim = optim.Adam(inference_network.parameters())
generative_network_optim = optim.Adam(generative_network.parameters())

num_iterations = 10
batch_size = 50
num_data = 100
generative_epochs_per_iteration = 100
inference_epochs_per_iteration = 10
num_generative_epochs = np.repeat(generative_epochs_per_iteration, num_iterations)
num_inference_epochs = np.repeat(inference_epochs_per_iteration, num_iterations)

(a, b, c) = (1, 0, 0)
data_generator = lambda: generative_model(a, b, c)
simple_dataset = SimpleDataset(infinite_data=False, num_data=num_data, data_generator=data_generator)
simple_dataloader = torch.utils.data.DataLoader(simple_dataset, batch_size=batch_size, shuffle=True)

generative_network_objective = []
inference_network_objective = []
if opt.visdom:
    util.vis.close()
    generative_network_objective_line = util.vis.line(
        X=np.array([0]), 
        Y=np.array([0]),
        opts=dict(
            xlabel='Epoch',
            ylabel='Objective (to maximize)',
            title='Generative network objective'
        )
    )
    
    inference_network_objective_line = util.vis.line(
        X=np.array([0]), 
        Y=np.array([0]),
        opts=dict(
            xlabel='Epoch',
            ylabel='Objective (to minimize)',
            title='Inference network objective'
        )
    )

for i in range(num_iterations):
    util.logger.info('Iteration {}'.format(i))
    
    # Step 1
    for epoch in range(num_generative_epochs[i]):
        util.logger.info('Generative network step | Epoch {}'.format(epoch))
        temp_generative_network_objective = []
        for _, y in enumerate(simple_dataloader):
            k, x = inference_network.sample(y.unsqueeze(1))
            
            generative_network_optim.zero_grad()
            logpdf_generative_network = generative_network.forward(Variable(k), Variable(x), Variable(y))
            temp_generative_network_objective += logpdf_generative_network.data.numpy().tolist()
            utility = torch.mean(logpdf_generative_network)
            loss = -utility # we want to maximize
            loss.backward()
            generative_network_optim.step()
        generative_network_objective.append(np.mean(temp_generative_network_objective))
        
        util.vis.line(
            X=np.arange(len(generative_network_objective)),
            Y=np.nan_to_num(np.array(generative_network_objective)),
            update='replace',
            win=generative_network_objective_line
        )
    
    # Step 2
    for epoch in range(num_inference_epochs[i]):
        util.logger.info('Inference network step | Epoch {}'.format(epoch))
        temp_inference_network_objective = []
        for batch_size in util.chunk(num_data, batch_size):
            k, x, y = generative_network.sample(batch_size)
            
            inference_network_optim.zero_grad()
            logpdf_inference_network = inference_network.forward(Variable(k), Variable(x), Variable(y))
            temp_inference_network_objective += logpdf_inference_network.data.numpy().tolist()
            loss = torch.mean(logpdf_inference_network)
            loss.backward()
            inference_network_optim.step()
        inference_network_objective.append(np.mean(temp_inference_network_objective))
        util.vis.line(
            X=np.arange(len(inference_network_objective)),
            Y=np.nan_to_num(np.array(inference_network_objective)),
            update='replace',
            win=inference_network_objective_line
        )

2017-07-19 16:09:36,847: Iteration 0
2017-07-19 16:09:36,848: Generative network step | Epoch 0
2017-07-19 16:09:36,860: Generative network step | Epoch 1
2017-07-19 16:09:36,870: Generative network step | Epoch 2
2017-07-19 16:09:36,881: Generative network step | Epoch 3
2017-07-19 16:09:36,892: Generative network step | Epoch 4
2017-07-19 16:09:36,903: Generative network step | Epoch 5
2017-07-19 16:09:36,914: Generative network step | Epoch 6
2017-07-19 16:09:36,925: Generative network step | Epoch 7
2017-07-19 16:09:36,936: Generative network step | Epoch 8
2017-07-19 16:09:36,947: Generative network step | Epoch 9
2017-07-19 16:09:36,958: Generative network step | Epoch 10
2017-07-19 16:09:36,969: Generative network step | Epoch 11
2017-07-19 16:09:36,979: Generative network step | Epoch 12
2017-07-19 16:09:36,991: Generative network step | Epoch 13
2017-07-19 16:09:37,001: Generative network step | Epoch 14
2017-07-19 16:09:37,012: Generative network step | Epoch 15
2017-07-19 16

2017-07-19 16:09:38,406: Generative network step | Epoch 26
2017-07-19 16:09:38,417: Generative network step | Epoch 27
2017-07-19 16:09:38,429: Generative network step | Epoch 28
2017-07-19 16:09:38,441: Generative network step | Epoch 29
2017-07-19 16:09:38,454: Generative network step | Epoch 30
2017-07-19 16:09:38,466: Generative network step | Epoch 31
2017-07-19 16:09:38,478: Generative network step | Epoch 32
2017-07-19 16:09:38,489: Generative network step | Epoch 33
2017-07-19 16:09:38,500: Generative network step | Epoch 34
2017-07-19 16:09:38,512: Generative network step | Epoch 35
2017-07-19 16:09:38,525: Generative network step | Epoch 36
2017-07-19 16:09:38,535: Generative network step | Epoch 37
2017-07-19 16:09:38,547: Generative network step | Epoch 38
2017-07-19 16:09:38,559: Generative network step | Epoch 39
2017-07-19 16:09:38,572: Generative network step | Epoch 40
2017-07-19 16:09:38,583: Generative network step | Epoch 41
2017-07-19 16:09:38,595: Generative netw

2017-07-19 16:09:40,057: Generative network step | Epoch 53
2017-07-19 16:09:40,069: Generative network step | Epoch 54
2017-07-19 16:09:40,081: Generative network step | Epoch 55
2017-07-19 16:09:40,094: Generative network step | Epoch 56
2017-07-19 16:09:40,106: Generative network step | Epoch 57
2017-07-19 16:09:40,119: Generative network step | Epoch 58
2017-07-19 16:09:40,131: Generative network step | Epoch 59
2017-07-19 16:09:40,143: Generative network step | Epoch 60
2017-07-19 16:09:40,155: Generative network step | Epoch 61
2017-07-19 16:09:40,167: Generative network step | Epoch 62
2017-07-19 16:09:40,180: Generative network step | Epoch 63
2017-07-19 16:09:40,192: Generative network step | Epoch 64
2017-07-19 16:09:40,204: Generative network step | Epoch 65
2017-07-19 16:09:40,216: Generative network step | Epoch 66
2017-07-19 16:09:40,229: Generative network step | Epoch 67
2017-07-19 16:09:40,241: Generative network step | Epoch 68
2017-07-19 16:09:40,254: Generative netw

2017-07-19 16:09:41,833: Generative network step | Epoch 80
2017-07-19 16:09:41,844: Generative network step | Epoch 81
2017-07-19 16:09:41,857: Generative network step | Epoch 82
2017-07-19 16:09:41,870: Generative network step | Epoch 83
2017-07-19 16:09:41,883: Generative network step | Epoch 84
2017-07-19 16:09:41,895: Generative network step | Epoch 85
2017-07-19 16:09:41,908: Generative network step | Epoch 86
2017-07-19 16:09:41,920: Generative network step | Epoch 87
2017-07-19 16:09:41,932: Generative network step | Epoch 88
2017-07-19 16:09:41,945: Generative network step | Epoch 89
2017-07-19 16:09:41,957: Generative network step | Epoch 90
2017-07-19 16:09:41,970: Generative network step | Epoch 91
2017-07-19 16:09:41,983: Generative network step | Epoch 92
2017-07-19 16:09:41,995: Generative network step | Epoch 93
2017-07-19 16:09:42,009: Generative network step | Epoch 94
2017-07-19 16:09:42,022: Generative network step | Epoch 95
2017-07-19 16:09:42,034: Generative netw

2017-07-19 16:09:43,593: Inference network step | Epoch 7
2017-07-19 16:09:43,607: Inference network step | Epoch 8
2017-07-19 16:09:43,619: Inference network step | Epoch 9
2017-07-19 16:09:43,631: Iteration 5
2017-07-19 16:09:43,632: Generative network step | Epoch 0
2017-07-19 16:09:43,646: Generative network step | Epoch 1
2017-07-19 16:09:43,659: Generative network step | Epoch 2
2017-07-19 16:09:43,673: Generative network step | Epoch 3
2017-07-19 16:09:43,685: Generative network step | Epoch 4
2017-07-19 16:09:43,698: Generative network step | Epoch 5
2017-07-19 16:09:43,711: Generative network step | Epoch 6
2017-07-19 16:09:43,723: Generative network step | Epoch 7
2017-07-19 16:09:43,735: Generative network step | Epoch 8
2017-07-19 16:09:43,749: Generative network step | Epoch 9
2017-07-19 16:09:43,762: Generative network step | Epoch 10
2017-07-19 16:09:43,775: Generative network step | Epoch 11
2017-07-19 16:09:43,788: Generative network step | Epoch 12
2017-07-19 16:09:43

2017-07-19 16:09:45,418: Generative network step | Epoch 24
2017-07-19 16:09:45,431: Generative network step | Epoch 25
2017-07-19 16:09:45,444: Generative network step | Epoch 26
2017-07-19 16:09:45,456: Generative network step | Epoch 27
2017-07-19 16:09:45,468: Generative network step | Epoch 28
2017-07-19 16:09:45,482: Generative network step | Epoch 29
2017-07-19 16:09:45,496: Generative network step | Epoch 30
2017-07-19 16:09:45,509: Generative network step | Epoch 31
2017-07-19 16:09:45,522: Generative network step | Epoch 32
2017-07-19 16:09:45,536: Generative network step | Epoch 33
2017-07-19 16:09:45,549: Generative network step | Epoch 34
2017-07-19 16:09:45,563: Generative network step | Epoch 35
2017-07-19 16:09:45,577: Generative network step | Epoch 36
2017-07-19 16:09:45,590: Generative network step | Epoch 37
2017-07-19 16:09:45,604: Generative network step | Epoch 38
2017-07-19 16:09:45,618: Generative network step | Epoch 39
2017-07-19 16:09:45,631: Generative netw

2017-07-19 16:09:47,234: Generative network step | Epoch 51
2017-07-19 16:09:47,247: Generative network step | Epoch 52
2017-07-19 16:09:47,261: Generative network step | Epoch 53
2017-07-19 16:09:47,275: Generative network step | Epoch 54
2017-07-19 16:09:47,290: Generative network step | Epoch 55
2017-07-19 16:09:47,304: Generative network step | Epoch 56
2017-07-19 16:09:47,317: Generative network step | Epoch 57
2017-07-19 16:09:47,331: Generative network step | Epoch 58
2017-07-19 16:09:47,344: Generative network step | Epoch 59
2017-07-19 16:09:47,358: Generative network step | Epoch 60
2017-07-19 16:09:47,371: Generative network step | Epoch 61
2017-07-19 16:09:47,385: Generative network step | Epoch 62
2017-07-19 16:09:47,399: Generative network step | Epoch 63
2017-07-19 16:09:47,412: Generative network step | Epoch 64
2017-07-19 16:09:47,426: Generative network step | Epoch 65
2017-07-19 16:09:47,441: Generative network step | Epoch 66
2017-07-19 16:09:47,455: Generative netw

2017-07-19 16:09:49,139: Generative network step | Epoch 78
2017-07-19 16:09:49,154: Generative network step | Epoch 79
2017-07-19 16:09:49,168: Generative network step | Epoch 80
2017-07-19 16:09:49,181: Generative network step | Epoch 81
2017-07-19 16:09:49,195: Generative network step | Epoch 82
2017-07-19 16:09:49,209: Generative network step | Epoch 83
2017-07-19 16:09:49,224: Generative network step | Epoch 84
2017-07-19 16:09:49,238: Generative network step | Epoch 85
2017-07-19 16:09:49,252: Generative network step | Epoch 86
2017-07-19 16:09:49,266: Generative network step | Epoch 87
2017-07-19 16:09:49,280: Generative network step | Epoch 88
2017-07-19 16:09:49,294: Generative network step | Epoch 89
2017-07-19 16:09:49,308: Generative network step | Epoch 90
2017-07-19 16:09:49,323: Generative network step | Epoch 91
2017-07-19 16:09:49,337: Generative network step | Epoch 92
2017-07-19 16:09:49,353: Generative network step | Epoch 93
2017-07-19 16:09:49,367: Generative netw

2017-07-19 16:09:51,069: Inference network step | Epoch 5
2017-07-19 16:09:51,080: Inference network step | Epoch 6
2017-07-19 16:09:51,092: Inference network step | Epoch 7
2017-07-19 16:09:51,103: Inference network step | Epoch 8
2017-07-19 16:09:51,113: Inference network step | Epoch 9


In [9]:
fig, ax = plt.subplots(nrows=1, ncols=2)
fig.set_size_inches(10, 4)
ax[0].plot(
    np.arange(len(generative_network_objective)),
    np.nan_to_num(generative_network_objective)
)
ax[0].plot(
    np.cumsum(num_generative_epochs) - 1,
    np.nan_to_num(np.array(generative_network_objective)[np.cumsum(num_generative_epochs) - 1]),
    linestyle='None',
    marker='x',
    markersize=5,
    markeredgewidth=1
)
ax[0].set_xlim([0, len(generative_network_objective) - 1])
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Objective (to maximize)')
ax[0].set_title('Generative network objective')

ax[1].plot(
    np.arange(len(inference_network_objective)),
    np.nan_to_num(inference_network_objective)
)
ax[1].plot(
    np.cumsum(num_inference_epochs) - 1,
    np.nan_to_num(np.array(inference_network_objective)[np.cumsum(num_inference_epochs) - 1]),
    linestyle='None',
    marker='x',
    markersize=5,
    markeredgewidth=1
)
ax[1].set_xlim([0, len(inference_network_objective) - 1])
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Objective (to minimize)')
ax[1].set_title('Inference network objective')

<IPython.core.display.Javascript object>